In [ ]:
import sys
sys.path.insert(0,'/workspace')
sys.path.insert(0,'/workspace/jazzstock_bot')

In [ ]:
import common.connector_db as db
import pandas as pd
pd.options.display.max_rows = 500
pd.options.display.max_columns= 500
pd.options.display.expand_frame_repr=False


In [ ]:
def get_relative_strength(date)# 종목 추출

        df = db.selectpd(f''' \


        SELECT STOCKCODE, DATE, 
        MC,
        (C.CLOSE + C.HIGH + C.CLOSE)/3 * INS AS IPA,   # 고저종평균 X 기관매수주식수 == 당일 기관순매수 금액
        (C.CLOSE + C.HIGH + C.CLOSE)/3 * FOREI AS FPA,
        (C.CLOSE + C.HIGH + C.CLOSE)/3 * PER AS PPA

        FROM jazzdb.T_STOCK_SND_DAY A
        JOIN jazzdb.T_STOCK_MC B USING (STOCKCODE, DATE)
        JOIN jazzdb.T_STOCK_OHLC_DAY C USING (STOCKCODE, DATE)
        JOIN jazzdb.T_DATE_INDEXED USING(DATE)
        JOIN jazzdb.T_STOCK_FUTURE_PRICE USING(STOCKCODE, DATE)
        JOIN jazzdb.T_STOCK_SND_ANALYSIS_RESULT_TEMP USING(STOCKCODE, DATE)

        WHERE 1=1
        AND DATE = '{date}'

        ''')

        # 한국상장종목내 시가총액 합 계산
        df['MCR']= df['MC']/df['MC'].sum()

        for win in ['I','F']:

            df_b = df[df[f'{WIN}PA']>0]
            df_b[F'{WIN}PA_SUM'] = df_b[f'{WIN}PA'].sum()
            df_b[f'{WIN}PAR']= df_b[f'{WIN}PA']/df_b[f'{WIN}PA_SUM']
            df_b[f'{WIN}PAR']= df_b[f'{WIN}PAR'].astype(float)
            df_b[f'{WIN}1R'] = df_b[f'{WIN}PAR'] / df_b['MCR']
            
            print(df_b)
        
    

In [ ]:
# 한국상장종목내 시가총액 합 계산
df['MCR']= df['MC']/df['MC'].sum()

In [ ]:
# 기관 매수종목 및 매도 종목으로 분리
df_b = df[df['IPA']>0]
df_s = df[df['IPA']<0]

In [ ]:
df_b['SUMIPA'] = df_b['IPA'].sum()
df_s['SUMIPA'] = df_s['IPA'].sum()

In [ ]:
df_b['IPAR']= df_b['IPA']/df_b['SUMIPA']

In [ ]:
df_b['IPAR']= df_b.IPAR.astype(float)
df_b['SUMIPA']=df_b['SUMIPA'].astype(float)

In [ ]:
# df_b.dtypes
df_b['RESPONSE_IDX'] = df_b['IPAR'] / df_b['MCR']

In [86]:
df_b.sort_values('RESPONSE_IDX', ascending= False)[['STOCKNAME','DATE','MC','IPAR', 'IPA', 'MCR','RESPONSE_IDX', 'PRO5', 'PRO3','I5']].round(5).head(100).reset_index(drop=True)

,STOCKNAME,DATE,MC,IPAR,IPA,MCR,RESPONSE_IDX,PRO5,PRO3,I5
0,원익홀딩스,2020-11-20,4.14768,0.01036,3741798883.3331,0.00014,73.39631,0.190,0.096,0.02792
1,네패스아크,2020-11-20,3.61434,0.00828,2988266950.0000,0.00012,67.26500,-0.048,-0.061,NaN
2,엘비세미콘,2020-11-20,4.70684,0.01059,3823355750.0000,0.00016,66.08668,0.104,0.047,0.02858
3,네패스,2020-11-20,7.41353,0.01436,5186077916.6666,0.00025,56.91317,0.019,-0.011,0.03159
4,서진시스템,2020-11-20,8.17245,0.01462,5276964400.0000,0.00028,52.53284,0.036,0.011,0.01028
5,이연제약,2020-11-20,4.56763,0.00811,2929690200.0000,0.00016,52.18312,-0.156,-0.126,-0.02339
6,와이지-원,2020-11-20,2.05456,0.00331,1194662726.6665,0.00007,47.30699,-0.022,0.037,0.02675
7,상상인이안제2호스팩,2020-11-20,0.07681,0.00011,39901995.0000,0.00000,42.26589,0.000,0.000,0.00612
8,케이씨씨글라스,2020-11-20,2.85995,0.00402,1453193250.0000,0.00010,41.33942,0.060,-0.006,0.01369
9,이수화학,2020-11-20,3.71536,0.00500,1803520999.9999,0.00013,39.49297,-0.086,-0.089,0.02090


In [ ]:
df_b.sort_values('IPA', ascending= False)[['STOCKNAME','MC','IPAR']].head(50).reset_index(drop=True)

In [ ]:
df['IPAR'] = df['IPA']/df['SUMIPA']
df['MCR']  = df['MC']/df['SUMMC']

In [ ]:
df.sort_values('IPAR',ascending =False)

In [ ]:
df.sort_values('IPA',ascending =False)